<h1> DYNAMIC MALWARE CLASSIFICATION</h1>

Supervised, classification, multi-class

### CONTENT INDEX

1. SET-UP
  - Report
2. EDA
  - Data types
  - Descriptive statistics
    - Plots, summary statistics 
  - Relationships between variables 
  - other...
  - Conclusion 
3. DATA SPLITTING
	- Encoding
4. Data preprocessing
	- Feature scaling
	- Duplicate data anlayis
  - Missing data analysis
  - Outlier detection analysis
  - Data imputing
  - Class imbalance
5. FEATURE ANALYSIS
  - Feature engineering
  - Feature selection
6. MODELLING
  - Non-optimized training
  - Optimized training
  - Conclusions mkdelling 

# 0. Basic Set-Up and General info

Note: there are a lot of non-used imports (to be removed at the end)

In [11]:
from collections import defaultdict
import math
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
from scipy import stats

If u make a change to utilities_function u may need to restart the kernel to use the latest versions. This is too tedious. There must be another way.

In [12]:
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                              BaggingClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [14]:
from imblearn.over_sampling import SMOTENC
from boruta import BorutaPy
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [13]:
import importlib

# Import your modules initially
import library.dataset
import library.modelling.shallow.regressor

# Reload the modules in case they were updated
importlib.reload(library.dataset)
importlib.reload(library.modelling.shallow.regressor)

# Now you can import specific classes/functions
from library.dataset import Dataset 
from library.modelling.shallow.classifier import ClassifierAssesment
from library.plots import Plots

## Documentation Standarization
Follow the following guidelines to make sure our document is consistent and easier to undestand
- All constant should be writte in full capitall letters (e.g: MY_CONSTANT). Notebook-level constants should be written in the cell assigned to it (see below)
- Every new major section (EDA, Feature engineering) should be written with '# My title'#
- Subsequent sections should use '##', '###' or '####' hierarchacically
- At the beginning of each major section write a content index with the content included in that section (see 'feature engineering' section as an example)
- Please write paragraphs before and after each cell explain what u are about to do and the conclusions, correspondingly. Do not assume they are too obvious.
- Avoid excessive ChatGPT-originated comments
- Avoid writing more than 20 lines per code cell (exceptions for subroutines, which should be written in utilitied_functions.py)
- Ideally, add a "Questions" and "Things to be done" section in each major sections where u write about futher iterations u want to do (while sharing in with the rest)


## Set-up

In [15]:
file_path ="./dataset/dynamic_dataset.csv"
results_path = "results/results.csv"

In [16]:
results_columns = ["model", "accuracy", "precision", "recall", "F1", "comment", "features_used", "hyperParameterOptimization", "hyperParameters","comments"]
columns_to_check_duplicates = ["model", "features_used", "hyperParameterOptimization", "hyperParameters"]

In [17]:
# Initializing all the libraries' classes
dataset = Dataset(file_path)
plots = Plots(dataset)
modelAssesment = ClassifierAssesment(dataset, results_path, results_columns, columns_to_check_duplicates)

In [ ]:
dataset.df.head()

In [10]:
dataset.df.rename(columns={"reboot": "Reboot"}, inplace=True) # consistency with other features' names
dataset.df.drop(columns=["Family", "Hash"], inplace=True) # We have decided to use only category as target variable; Hash is temporary while im debugging (it will be deleted in EDA)

In [ ]:
dataset.get_basic_info(print_info=False)

### Notebook-level constants

In [32]:
ONLY_NUMERICAL_COLUMNS = dataset.df.select_dtypes(include='number')
RANDOM_STATE = 99

### Description of our features

For each of our entries in the dataset we have a feature describing the state of the (operating) system under which the malware was running. Here is an intial description for each of them.
Please access this document for a complete problem context description: https://docs.google.com/document/d/1yH9gvnJVSH9GLv9ATQ5JQWA2z8Jy4umxxRfMF-y2fiU/edit?usp=sharing

# 1. EDA

In this section, we conduct an Exploratory Data Analysis (EDA) on the dynamic malware dataset to gain initial insights into the structure, distribution, and quality of the data prior to modeling. The dataset includes behavioral features extracted from Android applications, along with labels indicating their respective malware categories. Understanding the composition of the dataset, such as class imbalance, feature correlations, and the presence of outliers, is crucial to ensure robust preprocessing, informed feature engineering, and  the success of machine learning classifiers.


### SUGGESTIONS FOR IMPROVEMENTS
- Cluster analysis

## Data Type Distribution

In [ ]:
# Get the data types of each column
data_types = dataset.df.dtypes

# Count the frequency of each data type
type_counts = data_types.value_counts()

# Plotting the frequency of data types
type_counts.plot(kind='bar', color='skyblue')

# Add title and labels
plt.title('Frequency of Data Types in DataFrame')
plt.xlabel('Data Type')
plt.ylabel('Frequency')


We can see that most columns are numerical. Lets gets to see which are the variables that are of type object.

In [ ]:
df_onlyCols = dataset.df.select_dtypes(include=["object"]).columns
df_onlyCols

## Summary Statistics Overview

## Histograms


In [ ]:
# Select only numerical features
numerical_cols = dataset.df.select_dtypes(include=[np.number]).columns

# Define number of rows and columns for subplots
num_features = len(numerical_cols)
cols = 4  # Number of columns per row
rows = math.ceil(num_features / cols)  # Calculate required rows

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(16, rows * 4))
axes = axes.flatten()  # Flatten to easily iterate

# Plot histograms
for i, col in enumerate(numerical_cols):
    sns.histplot(df[col], bins=30, kde=True, ax=axes[i])  # kde=True for smooth curve
    axes[i].set_title(col)

# Remove empty subplots
for i in range(num_features, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

We can see most distributions tend to be right-skewed and only a small portion follows a normal distribution. This right-skewness will be dealt in feature-engineering.

In [ ]:
# Select only numerical features
numerical_cols = dataset.df.select_dtypes(include=[np.number]).columns

# Define number of rows and columns for subplots
num_features = len(numerical_cols)
cols = 4  # Number of columns per row
rows = math.ceil(num_features / cols)  # Calculate required rows

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(16, rows * 4))
axes = axes.flatten()  # Flatten to easily iterate

# Plot boxplots
for i, col in enumerate(numerical_cols):
    sns.boxplot(x=dataset.df[col], ax=axes[i])  # Boxplot for each feature
    axes[i].set_title(col)

# Remove empty subplots
for i in range(num_features, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

## Numerical Features

Seemed to be grouped by prefixes: Memory, Network, Battery, Logcat, Process y API.

According to dataset authors to capture how various malware families and categories behave at runtime, the analysis relies on six distinct sets of features obtained after executing each sample within a controlled emulated environment. These feature groups offer a comprehensive view of the malware's dynamic activity.

This categories appear before the first _ in every feature label and are defined as:


"Memory: Memory features define activities performed by malware by utilizing memory.

API: Application Programming Interface (API) features delineate the communication between two applications.

Network: Network features describe the data transmitted and received between other devices in the network. It indicates foreground and background network usage.

Battery: Battery features describe the access to battery wakelock and services by malware.

Logcat: Logcat features write log messages corresponding to a function performed by malware.

Process: Process features count the interaction of malware with total number of processes."



In [ ]:
numeric_cols = dataset.df.select_dtypes(include='number').columns

# Grouping based on the first prefix before "_"
prefix_groups = defaultdict(list)

for col in numeric_cols:
    prefix = col.split("_")[0]  # Get the first word before the underscore
    prefix_groups[prefix].append(col)

for prefix, columns in prefix_groups.items():
    print(f"\n {prefix} ({len(columns)} features):")
    for col in columns:
        print(f"  - {col}")

## Categorical Features

In [ ]:
#Statistical summary for categorical features
dataset.df.describe(include=["object", "category", "bool"])

In [ ]:
print(dataset.df[['Hash', 'Category', 'Family']].head())

Hash: unique identifier that represents each malware sample. <<<>>>THIS IS PROBABLY WRONG<<<>>>

Category: general classification of the malware sample based on its behavior.

Family: more fine-grained grouping of malware based on its codebase or origin

For hash, it will first be checked if the same malware before and after reboot contains the same hash value.

In [ ]:
# Count how many times each hash appears in 'before' and 'after'
hash_reboot_counts =dataset.df.groupby(['Hash', 'reboot']).size().unstack(fill_value=0)

# Hashes in both with exactly one in each
hashes_with_one_each = hash_reboot_counts[
    (hash_reboot_counts['before'] == 1) & (hash_reboot_counts['after'] == 1)
].index

# Hashes in both but with extra rows
hashes_in_both_but_not_clean = hash_reboot_counts[
    (hash_reboot_counts['before'] > 0) &
    (hash_reboot_counts['after'] > 0) &
    ~((hash_reboot_counts['before'] == 1) & (hash_reboot_counts['after'] == 1))
].index

# Total unique hashes
total_unique_hashes = dataset.df['Hash'].nunique()

# Hashes in only one reboot condition
hashes_in_one_condition = hash_reboot_counts[
    (hash_reboot_counts['before'] == 0) | (hash_reboot_counts['after'] == 0)
]

# Only once in one reboot condition
only_once_in_one = hashes_in_one_condition[
    (hashes_in_one_condition['before'] == 1) | (hashes_in_one_condition['after'] == 1)
]

# More than once in one reboot condition
more_than_once_in_one = hashes_in_one_condition[
    ((hashes_in_one_condition['before'] > 1) & (hashes_in_one_condition['after'] == 0)) |
    ((hashes_in_one_condition['after'] > 1) & (hashes_in_one_condition['before'] == 0))
]

# Split those into counts
more_than_once_in_before = more_than_once_in_one[more_than_once_in_one['before'] > 1]
more_than_once_in_after = more_than_once_in_one[more_than_once_in_one['after'] > 1]

# --- PRINT RESULTS ---
print(f"Hashes with EXACTLY one row in BOTH before and after: {len(hashes_with_one_each)}")
print(f"Hashes in BOTH, BUT with extra rows: {len(hashes_in_both_but_not_clean)}")

print(f"\nHashes in ONLY ONE reboot condition:")
print(f"• Appearing ONLY ONCE: {len(only_once_in_one)}")
print(f"• Appearing MORE THAN ONCE: {len(more_than_once_in_one)}")
print(f"   - More than once in BEFORE: {len(more_than_once_in_before)}")
print(f"   - More than once in AFTER: {len(more_than_once_in_after)}")

print(f"\nTotal breakdown:")
print(f"• In BOTH (any): {len(hashes_with_one_each) + len(hashes_in_both_but_not_clean)}")
print(f"• In ONLY ONE reboot: {len(hashes_in_one_condition)}")
print(f"• TOTAL unique hashes: {total_unique_hashes}")


A total of 19,169 hashes appear exactly once in both before and after conditions. These are highly reliable for paired  comparisons, ideal for understanding how reboot affects malware behavior.


There are 158 hashes that appear in both reboot states but not exactly once in each. These extra instances may come from inconsistencies in data capture like multiple logs for the same sample and should be checked.

A significant portion of samples appear only in one reboot condition. This is consistent with limitations described in the original dataset paper, where some malware samples failed to execute after the reboot. However, what is curious is that some still have been logged more than once.


In [ ]:
dataset.df.drop(columns=['Hash'], inplace=True)
'''
The Hash column is a high-cardinality feature, containing unique values for a high number of rows in the dataset.
It serves as an identifier for each malware sample. Including this column in modeling
would not only offer no predictive value but could also lead to overfitting or cause issues with algorithms that are
sensitive to high-cardinality categorical features.
 <<<>>> J.N: may be better to focus the argumentation on ID not being useful rather than high-cardinality per se. Also write the 
  argumentation in a text cell not in this type of comments. <<<>>>
'''

This research will be using both Category and Family as the target variables for classification.

## Reboot Analysis

In [ ]:
print(dataset.df["reboot"].value_counts())

The imbalance observed in the dataset, with 28,380 samples collected before reboot and only 25,059 after reboot, is explained by limitations found during the dynamic analysis. The authors of the dataset note that "there was no entry point in some Android malware samples and some Android malware samples stopped abruptly." This means that certain malware applications either failed to launch or terminated unexpectedly during execution, preventing the collection of dynamic behavior data, particularly after the reboot phase.

Additionally, the study highlights another critical limitation: "the dynamic analysis is performed in an emulator. Some malware samples are able to detect the emulated environment and are not executed." This behavior reflects common anti-analysis techniques used by sophisticated malware, which can detect when they are running in a sandbox or emulator and intentionally suspend their malicious actions.




<<<>>>THIS ANALYSIS IS SUPER GOOD (you can delete this comment)<<<>>>

The displayed features are the top 10  most affected by reboot showing a clear reboot-sensitive behavior.

In [ ]:
#Category distribution across reboot
plt.figure(figsize=(12, 6))
sns.countplot(data=dataset.df, x='Category', hue='reboot')
plt.title("Malware Categories by Reboot Condition")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


To identify which numeric features are most influenced by the reboot condition, the dataset will be grouped by the reboot variable, separating entries collected before and after the device reboot. Within each group, the mean of every numeric feature will be computed, allowing for the comparison of average behavior across both states.

A new column labeled 'diff' was then added, representing the difference between the mean values after and before the reboot for each feature. A positive value indicates that the feature increased after reboot, while a negative value shows it decreased.

In [ ]:
reboot_means = dataset.df.groupby('reboot').mean(numeric_only=True).T
reboot_means['diff'] = reboot_means['after'] - reboot_means['before']
reboot_means_sorted = reboot_means.sort_values(by='diff', ascending=False)

reboot_means_sorted.head(10)

The results reveal that several features show clear shifts after reboot. Specially, network-related features such as Network_TotalReceivedBytes and Network_TotalTransmittedBytes demonstrate significant increases, suggesting that some malware types intensify data transmission once the device has rebooted. Memory features like Memory_SharedClean, Memory_HeapSize, and Memory_HeapAlloc also show increased values after reboot, indicating greater memory use or altered memory management after reboot.
This shows that the reboot condition plays an important role in runtime behavior and should be treated as an important factor in exploratory analysis and modeling.

## Family

In [ ]:
#How many categories each family belongs to
dataset.df.groupby("Family")["Category"].nunique().sort_values(ascending=False)

Almost every family is either unknown or unique


In [378]:
# <<<Error: NameError: name 'family_to_category' is not defined>>> (this Irina's code; copied from Argentinan guy's notebook)
# multi_cat_families = family_to_category[family_to_category > 1]
# print(f"Number of families mapping to multiple categories: {len(multi_cat_families)}")
# print(multi_cat_families)

There is only one Family that maps to multiple categories, and is the placeholder unknown.

The following code displays how many samples with unknown family labels belong to each malware category.

In [ ]:
dataset.df[dataset.df["Family"] == "<unknown>"]["Category"].value_counts()

In [ ]:
# Step 1: Count unique families per category
family_amount = dataset.df.groupby("Category")["Family"].nunique()

# Step 2: Total number of instances per category
total_per_category = dataset.df["Category"].value_counts()

# Step 3: Count how many of those are <unknown> per category
unknown_amount = dataset.df[dataset.df["Family"] == "<unknown>"]["Category"].value_counts()

# Step 4: Combine all stats into a summary table
summary_df = pd.DataFrame({
    "Family_amount": family_amount,
    "Total_category": total_per_category,
    "Unknown_amount": unknown_amount
}).fillna(0).astype({"Unknown_amount": int})

# Step 5: Calculate percentage of unknowns per category
summary_df["%_Unknown"] = (summary_df["Unknown_amount"] / summary_df["Total_category"] * 100).round(2)

# Reorder columns for readability
summary_df = summary_df[["Family_amount", "Total_category", "Unknown_amount", "%_Unknown"]]

# Display the summary
print(summary_df)

In [ ]:
unknown_count = (dataset.df["Family"] == "<unknown>").sum()
print(f"Number of rows with Family == '<unknown>': {unknown_count}")


Based on the analysis of family distribution across categories:

The Adware category stands out with zero instances labeled as <unknown> and a balanced distribution across 43 families. This makes it a strong candidate for modeling.

In contrast, Zero_Day and No_Category The categories Zero_Day and No_Category exhibit extremely high family dispersion, with 2576 and 335 unique families. These values are significantly higher than all other categories, which generally have fewer than 50 families each.


This suggests they function more as placeholder labels. In particular, Zero_Day likely serves as a catch-all label for unknown or uncategorized threats, making it ambiguous. In cybersecurity, this term is refered to a new unknown vulnerability, not yet classified in terms of malware behavior, this is why samples are varied. They do not seem to represent a consistent type. On the other hand, No_Category explicitly denotes a lack of category. So, including these instances would only bring noise to the training process, preventing the model from learning meaningful patterns.
Therefore, they are excluded from the final dataset to preserve the quality and consistency of the classification task.


Additionally, categories like FileInfector show a high percentage of <unknown> families (6.85%) despite having a small total count, raising concerns about label quality. Most other categories maintain a relatively stable level of unknowns (around 3–5%), indicating that the presence of <unknown> is manageable.

# DATA SPLITTING
### TO BE DONE
- Statistical analysis of this
- Make sure they follow the same distributions

### Data Splitting: Category as target variable
Originally, we will focus only on category

Lets first get the X and y extracted from our dataset

Also object!
Lets get back to the splitting!

Before, we split the dataset lets observe the SE of accuracy variation based on our choice of split. 
Brief explanation: we can model accuracy via a Binomial distribution. We know each event in a binomial distribution can be modelled through a bernoulli distribution, where the outcome represents the probability predicting the correct class or not. We make the assumption that each classification error is independent from each other. For:
$$
\text{Bin} \sim (n, p)
$$
 let us assume that the parameter of this distribution is p = .85 and n is given by the choice of sample split for the test set. The SE of the sample proportion can be modeled via: 
$$
\text{SE}_{\hat{p}} = \sqrt{\frac{p(1 - p)}{n}}

$$

Before we continue to assess all possible choices of split based on a variant n, also note that that choice of evenly distributed split (e.g: 10% for each hold-out set) between hold-out sets is arbitrary. Proper choice is that which guarantees an equivalent distribution at each hold-out sets, which may not neccesarily be the equivalent split. This brings a new choice of tradeoff between certainity of prediction accuracy (higher test size, smaller validaiton set) but possibly less space for proper hyperparemter optimization or the inverse (higher validation size, smaller set set). As with other tradeoffs, the priority for a given option is rooted in the model's application (which may come derived from a client/employer). For our case, we dont favor either option of the tradeoof, thus we will keep the even hold-out distribution.

Finally, also note that that choice of evenly distributed split between hold-out sets is arbitrary. Proper choice is that which guarantees an equivalent distribution at each hold-out sets, which may not neccesarily be the equivalent split. This brings a new choice of tradeoff between certainity of prediction accuracy (higher test size, smaller validaiton set) but possibly less space for proper hyperparemter optimization or the inverse (higher validation size, smaller set set). As with other tradeoffs, the priority for a given option is rooted in the model's application (which may come derived from a client/employer). For our case, we dont favor either option of the tradeoof, thus we will keep the even hold-out distribution.


In [ ]:
dataset.asses_split_classifier(p=.85, step=.05, plot=True)

[We can see a diminishing-returns class graph](https://en.wikipedia.org/wiki/Knee_of_a_curve). The more we decline the training set percentage the slower and more steadier the current SE varies as well as the difference to prior SE, in percentage. We can see that the knee in the curve is between 80 and 90 training set percentage. This represents the area where when you start going below the lower bound, no **significant** improve appears. Given the fact that our criteria for choice of split percentage is to keep as much training possible while increasing hold-out sets size only if the decrease in SE is significant **we are going to select 80% for the training set**.

In [19]:
dataset.split_data(y_column="Category", train_size=.8, validation_size=.1, test_size=.1)

(       Memory_PssTotal  Memory_PssClean  Memory_SharedDirty  \
 48143            35115             2936               12944   
 10388            71418            11352                8960   
 11029           120450            30872               10704   
 36609            42669             4156               12544   
 26413            65679             3256               10792   
 ...                ...              ...                 ...   
 42697            35571              208               11060   
 36008            52445             5196               12452   
 46265            37220             3464               12612   
 23587           113947            39996               10852   
 29313            79501             5476               10600   
 
        Memory_PrivateDirty  Memory_SharedClean  Memory_PrivateClean  \
 48143                25560               83552                 3072   
 10388                46700               84844                11468   
 11029        

In [ ]:
dataset.X_train.shape, dataset.X_validation.shape, dataset.X_test.shape, dataset.y_train.shape, dataset.y_validation.shape, dataset.y_test.shape

## FEATURE ENCODING

#### Encoding the X matrices (one-hot encoder)
Lets fit the X encoder for the object column (reboot)
#### Encoding y column vector (labeller encoding) 
We know our models needs the target variable to be numerical. Lets transform this series object then!
We will fit the encoder in the training set, and extend it to the other sets

In [ ]:
dataset.get_dataset_encoded(encode_y=True)

Lets visualize the results of the encoding...

### Features

In [ ]:
dataset.X_train_encoded

In [ ]:
dataset.X_val_encoded

In [ ]:
dataset.X_test_encoded

### Target variable

In [ ]:
dataset.y_train_encoded,  "-"*30, dataset.y_val_encoded,  "-"*30,  dataset.y_test_encoded,  "-"*30, dataset.encoding_map

<hr>

# DATA PREPROCESSING

### Feature Scaling
We would usually use normalization when we're dealing with some sort of Gaussian Distribution and, on the other hand, would use standarization if we're dealing with some kind of normal distribution. We're going to figure out the distributions of the fields to determine which of the 2 is more appropriate.

First, we're going to test for scale. If we see a lot of difference between scales, we'll go for normalization.



<hr>

# FEATURE ANALYSIS

Here we will take adjust the features that compose the learning inputs to our model. The correctness of this section is pivotal for proper learning by the model

INDEX OF THIS SECTION:
- Feature engineering
- Feature selection

QUESTIONS FOR THIS SUBECTION


## FEATURE ENGINEERING
- Domain-specific features
- Binning
- Interaction terms

NOTE: This requires further understanding of the concepts around our problem's context. This will be deferred for a second iteration.

## FEATURE SELECTION
- Analyze correlation and low-variances


#### 1.1) Eliminating low-variances features
We will start off feature selection by analyzing the variables that have low variances. Features with low variances provide little new information for the model to learn from, thus they could introduce statistical noise. Due to this reason, they should be elimanted from the dataset. The reason why we don't focus on high-variance is because this symbolizes outliers, which have been dealt with before in data preporcesing.

We will start off this analysis eliminating univariate features (i.e: the featuers with constant values)

In [ ]:
original_number_of_features = dataset.df.shape[1]
zero_variance_features = ONLY_NUMERICAL_COLUMNS.std() == 0
if zero_variance_features.any():
    print("Zero-variance features found:")
    print(zero_variance_features[zero_variance_features].index)
    dataset.df.drop(columns=zero_variance_features[zero_variance_features].index, inplace=True)
    print(f"Removed {original_number_of_features - dataset.df.shape[1]} features with zero variance")
    ONLY_NUMERICAL_COLUMNS = dataset.df.select_dtypes(include='number') #updating global variable
else:
    print("No zero-variance features found.")

Lets focus on features with too few variance

In [ ]:
standard_deviations = ONLY_NUMERICAL_COLUMNS.std()
plt.figure(figsize=(12, 8))
plt.hist(standard_deviations, bins=30, edgecolor='black')
plt.title('Distribution of Standard Deviations for Numeric Features')
plt.xlabel('Standard Deviation')
plt.ylabel('Frequency')
plt.show()

As explained in scikit-learn's in (1.13.1 Feature selection)[https://scikit-learn.org/stable/modules/feature_selection.html], the variance threshold must be selected carefully. Too low may delete few variables, and too may be too restrictive, deleting more variables than it should.

In [ ]:
STD_THRESHOLD = .4

In [ ]:
cols_to_keep = standard_deviations[(standard_deviations < STD_THRESHOLD)].index
print(f"Removed {len(standard_deviations) - len(cols_to_keep)} features with low variance")

df = dataset.df[cols_to_keep]
df.shape


#### 1.2) Eliminating highly correlated feature
Highly correlated variables (multicolinearity) are problem for models because they introduce a redundancy (features that contain significantly related similar information are not bringing much new insight into the model's input) to the model that can introduce significant variance. This is due to the fact that small changes in the data may make the coefficeints of the highly correlated variables **swing** more than it should

In [ ]:
plots.plot_correlation_matrix(size="l")

A note on the shape of this heatmap: due to the high amount of features, and the redundancy to measure the correlation between features (where corr(A,B) = corr(B,A)) we set 'np.triu(np.ones_like(corr, dtype=bool))' in the utilities functions in order to show only new non-redudant correlations between features, thus the right triangle shape.

Well, thats a lot to digest! We can see some solid red (high positive correlation) and medium-solid blue (some high negative correlation). 
Lets use a non-visual methodology to confirm our initial hypothesis. We will use variance inflaction factor (VIF) along with checking manually.
A brief explanation on VIF. 
Formula: 



$$
VIF_i = \frac{1}{1 - R_i^2}
$$


You regress (i.e: do linear regression) on the ith feature as target variable, and all other features as predictors. You then compute the coefficient of determination as a way to measure how well the predictors fit the target variable. VIF values ranges from [1, inf], where lower bound signifies little multicolinearity (R^2 = 0), and upper bound occurs when R^2=1 (perfect multicolineairty). 5 is considered a standard threshold for VIF as it symbolized an 80% R^2.
Once you ve obtained the results of VIF, you need to delete the variable with the highest VIF, and recompute VIF until there is no multicolinearity. For example, say there are three features that are 4 features, 3 of them being linear combinations of each other. You would delete the variables with VIF until there is no VIF (when only one of those linear combinations remains). You cant delete all n-1 variables where n are the amount of variables with exceeding (with respect to the chosen threshold), because you may be deleting one that has high VIF due to another feature.

A relevant note on why one-hot-encoding must be done dropping the first one:
If we were to not remove one of the labels of one hot encoding, you would be able to predict which level of the categorical variable based on all other ones (there is only degree of freedom for categories levels; they are a linear combination). You would essentially see inf VIF in that area and delete it in this section. 
The VIF has to be computed every time we delete a feature due to high multicolinearity. Lets do that

In [17]:
number_of_features = ONLY_NUMERICAL_COLUMNS.shape[1]

In [ ]:
ONLY_NUMERICAL_COLUMNS

In [18]:
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
    return vif_data

# Iteratively remove features with high VIF (threshold = 10)
def remove_high_vif_features(df, threshold=8):
    number_of_iterations = 0
    while True:
        number_of_iterations += 1
        vif_data = calculate_vif(df)
        print(f"VIF computed for iteration {number_of_iterations}:")
        max_vif = vif_data["VIF"].max()
        if max_vif < threshold:
            break
        feature_to_drop = vif_data.loc[vif_data["VIF"].idxmax(), "Feature"]
        df.drop(columns=[feature_to_drop], inplace=True)
        print(f"Dropped: {feature_to_drop}")
    return df

In [ ]:
# Note, running this cell will take a while
remove_high_vif_features(ONLY_NUMERICAL_COLUMNS, 10)
ONLY_NUMERICAL_COLUMNS = df.select_dtypes(include='number')

In [ ]:
ONLY_NUMERICAL_COLUMNS = df.select_dtypes(include='number')
print(f"Removed {df.shape[1] - number_of_features} features")

Lets compute the correlation matrix one more time post-deletion

In [ ]:
plots.plot_correlation_matrix(size="l")

### 1.3 Automatic Feature Selection
#### L1 regularization
Because of L1 regularization being able to set weights 0, we can briefly train our logistic regression model with such regulartization and see which features it uses. The reason why it sets 0 to insignificant feature is because the objective function is not only the MSE but added a component of the wieght magnitude (which is trying to minimize)

In [ ]:
model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1)
ModelAssesment.automatic_feature_selection_l1(model)

#### BORUTA
Brief explanation: (to be done)
Sources: (Tree-based feature selection)[https://scikit-learn.org/stable/modules/feature_selection.html#:~:text=1.13.4.2.%20Tree%2Dbased%20feature%20selection,-%23]


In [ ]:
rf = RandomForestClassifier(
    n_estimators=100,    
    n_jobs=-1, # multithreading
    class_weight='balanced',
    random_state=RANDOM_STATE
)
boruta_selector = BorutaPy(
    rf,
    n_estimators='auto',
    verbose=2,
    random_state=RANDOM_STATE,
    max_iter=10
)

In [ ]:
predictivePowerFeatures, excludedFeatures = ModelAssesment.automatic_feature_selection_boruta(boruta_selector)

In [ ]:
dataset.eliminate_features_from_all_sets(excludedFeatures)

Boruta said that only 61 variables were important, if that were the case the current dimensions for each set would have that nubmer

In [ ]:
dataset.X_train_encoded.shape, dataset.X_val_encoded.shape, dataset.X_test_encoded.shape

Awesome, lets move on!

<hr>

# MODELLING

- QUESTION ON THIS SECTION
   - How can statistical measurement help on the split?
- To be done
   -  as

## Fitting the model

QUESTIONS FOR THIS SECTION
- Can the ROC curve be used for multiclass?
- Can a unsupervised learning algorithm (e.g: KNN) be used for this problem even tough its nature is to be supervised?

TO BE DONE FOR THIS SECTION
- Multiple models as classifiers
- Learn more about each alogrithms' paremters
- Can KNN be used here?

## Random Forest & Decision Trees
Instead of the originally planned logistic regression, we will be using an ensembled model first: random forest (collection of week decision trees). This model is not only likely to outperform the original choice because its nature to handle multiclass better, but also does this several orders of magnitude faster. We will add its not ensembled version too, along with gradient boosted machine
Note we are not using not-by-default multiclass classifiers (e.g: logistic regressions, svms)

## Non-optimized fitting

In [12]:
randomForestModel = RandomForestClassifier(random_state=RANDOM_STATE, 
                             n_estimators=100,
                             max_depth=None,
                             min_samples_split=2, 
                             min_samples_leaf=1)

decisionTreeModel = DecisionTreeClassifier(
                                          random_state=RANDOM_STATE)

supportVectorModel = SVC(random_state=RANDOM_STATE,
             kernel='rbf',
             decision_function_shape='ovr',
             class_weight='balanced')

logisticRegressionModel = LogisticRegression(penalty='l2', 
                                             solver='sag',
                                             max_iter=1000,
                                             class_weight='balanced')

#### MODEL PERFORMANCE, HYPOTHESIS
- Trees (e.g: random forest, decision trees):
  - Time to fit:
    - Ensemble models (random forest) take more time to train due to the fact that they are larger and heavier than their non-ensembled version. 
  - Correctness:
     - High
- Binary-classifiers by default models (e.g: SMV, logistic regression)
  - Time to fit:
    - compute C models for all C number of classes. Each trained to detect a single class, then when we make predictions, we select the ones that has the highest probability in its predictions ("the most confident in its prediction"). This strategy is called One-vs-Rest, note however, a single logistic regression may be used if we used the softmax objective function (instead of log-odds) (it still heavy computationally, tough). They will be sloder than ensemble models
  - Correctness:
     - Very low if the problem is non-linear which it is for the SVM and logistic regression

In [ ]:
modelAssesment.add_model("Random Forest", randomForestModel)
modelAssesment.add_model("Decision Tree", decisionTreeModel)
modelAssesment.add_model("SVM", supportVectorModel)
modelAssesment.add_model("Logistic Regression", logisticRegressionModel)


Lets fit the models

In [ ]:
modelAssesment.fit_models(print_results=True, modelsToExclude=["Logistic Regression", "SVM"]) # exlcluding these models cause they are too slow to fit locally

Let's make sure the predictions vary between holdout sets

In [ ]:
modelAssesment.constrast_sets_predictions()

<i> the aforeshown diagram was originally done with the sole intention to debug an error that made predictions be the same across sets. Insights may not be much meaningful after correctio, but it is worth keeping until the end of the notebook development </i>

### PREDICTIONS RESULTS 
Before we get into the actual results, lets elaborate briefly on all the metrics that we are using to asses our classifiers:

- Accuracy => total correctly predicted elemetnts (sigma over the moments we predicted x_i and it was actually x_i / number_of_samples)
$$
\text{Accuracy} = \frac{\sum_{i} \mathbf{1}(\hat{y}_i = y_i)}{N}
$$
- Precision => out of how many predicted for that class were actually from that class (predicted for class x when it was x/ predicted for class x when it was x + predicted for class x when it was NOT x)
$$
\text{Precision}_x = \frac{\text{TP}_x}{\text{TP}_x + \text{FP}_x}
$$
- Recall => out of all cases that were positive how many got predicted correctly?
$$
\text{Recall}_x = \frac{\text{TP}_x}{\text{TP}_x + \text{FN}_x}
$$
- F1-score => harmonic mean of precision and recall (balances both metrics, heavily penalize spreadness between ratios that are being averaged out)
$$
\text{F1}_x = 2 \times \frac{\text{Precision}_x \times \text{Recall}_x}{\text{Precision}_x + \text{Recall}_x}
$$
- Support => number of actual occurences of class in the dataset
- macro avg => averages given metric across all classes
$$
\text{Macro Avg} = \frac{1}{C} \sum_{i=1}^{C} M_i
$$
- weighted avg => averages with weights per class occurence (considers frequency of class in average computation)
$$
\text{Weighted Avg} = \sum_{i=1}^{C} \frac{\text{Support}_i}{\text{Total Instances}} \times M_i
$$

In [ ]:
modelAssesment.evaluate_classifier(plot=True)

In [ ]:
modelAssesment.models

## Hyperparameter optimization

Note 1: the following cells took 11mins to run in an M2
Note 2: grid-search is not a good choice for optimization (enough for an early iteration, tough). We need to get smarter using more advanced techniques (e.g: OPTUNA)

In [ ]:
# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE), param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(dataset.X_category_train_encoded, dataset.y_category_train_encoded)

# Best model
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

In [27]:
best_model.fit(dataset.X_category_train_encoded, dataset.y_category_train_encoded)
y_val_pred = best_model.predict(dataset.X_category_val_encoded)
y_test_pred = best_model.predict(dataset.X_category_test_encoded)


## NOTEBOOK, AREAS FOR IMPROVEMENTS
- Introducing as many ML algorithms for non-predictive tasks: 
  - Data imputing with KNN 
  - Cluster analysis to better understand the features 
  - Doing PCA for features (after correlation analysis) 

- Distilled models
- DevOPs pipeline design
- Deeper statistical approach to data preprocessing and EDA
- Using CCE and BCE (deep learning multi-class metrics) used for shallow learning 